In [ ]:
# baseline.ipynb
import sys, os

PROJECT_ROOT = "/content/cnn-fairness-tradeoff/project"

COMMON_PATH = os.path.join(PROJECT_ROOT, "common")
sys.path.append(COMMON_PATH)

from dataset import load_imbalanced_cifar
from model import SimpleCNN

from pipeline.train import train_model
from pipeline.eval import eval_model
from utils import plot_cm

from utils import get_device, set_seed

from torch.utils.data import DataLoader, random_split

set_seed(42)
device = get_device()
print("Device:", device)

# Load data
dataset = load_imbalanced_cifar(cat_count=30, dog_count=500)
train_ds, test_ds = random_split(
    dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)]
)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

# Model
model = SimpleCNN().to(device)
model, losses = train_model(model, train_loader, device, epochs=5)

# Evaluation
overall, cat_acc, dog_acc, cm = eval_model(model, test_loader, device)
print("Overall:", overall)
print("Cat acc:", cat_acc)
print("Dog acc:", dog_acc)

plot_cm(cm)


# 📊 ⬇️ שמירת תוצאות בשיטס (גיליון baseline)
metrics = {
    "overall_acc": overall,
    "cat_acc": cat_acc,
    "dog_acc": dog_acc,
}

config = {
    "method": "baseline",
    "epochs": 5,
    "batch_size": 32,
    "lr": 1e-3,
    "cat_count": 30,
    "dog_count": 500,
}

log_experiment_to_sheet(
    experiment_name="baseline",
    metrics=metrics,
    config=config,
    notes="Imbalanced CIFAR-10: 30 cats vs 500 dogs, simple CNN, no mitigation.",
)